In [1]:
import os
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.utils import AnalysisException

In [2]:
spark = (SparkSession
         .builder
         .getOrCreate())

In [3]:
DIRECTORY = "../../data/broadcast_logs"
# FILE = "BroadcastLogs_2018_Q3_M8.CSV"
FILE = "BroadcastLogs_2018_Q3_M8_sample.CSV"

In [4]:
logs = spark.read.csv(
    path=os.path.join(DIRECTORY, FILE),
    sep="|",
    header=True,
    inferSchema=True,
    timestampFormat="yyyy-MM-dd",
    )

In [5]:
def split_table(df, sections, records):
    col_split = np.array_split(np.array(logs.columns), sections)
    for x in col_split:
        df.select(*x).show(records, False)

In [6]:
split_table(logs, 3, 3)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+
|BroadcastLogID|LogServiceID|LogDate   |SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|
+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+
|1196192316    |3157        |2018-08-01|1         |4                  |NULL                  |13        |3              |3                |NULL            |
|1196192317    |3157        |2018-08-01|2         |NULL               |NULL                  |NULL      |1              |NULL             |NULL            |
|1196192318    |3157        |2018-08-01|3         |NULL               |NULL                  |NULL      |1              |NULL             |NULL            |
+--------------+------------+----------+----------+-------

In [7]:
logs.select(F.col("Duration")).dtypes

[('Duration', 'string')]

In [8]:
logs.select(
    F.col("Duration"),
    F.col("Duration").substr(1, 2).cast("int").alias("dur_hours"),
    F.col("Duration").substr(4, 2).cast("int").alias("dur_minutes"),
    F.col("Duration").substr(7, 2).cast("int").alias("dur_seconds"),
    ).show()

+----------------+---------+-----------+-----------+
|        Duration|dur_hours|dur_minutes|dur_seconds|
+----------------+---------+-----------+-----------+
|02:00:00.0000000|        2|          0|          0|
|00:00:30.0000000|        0|          0|         30|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|         15|
|00:00:30.0000000|        0|          0|         30|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|         15|
|00:00:30.0000000|        0|          0|         30|
|00:00:30.0000000|        0|          0|         30|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|         15|
|00:00:15.0000000|        0|          0|      

El df sigue sin modificaciones:

In [9]:
split_table(logs, 3, 3)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+
|BroadcastLogID|LogServiceID|LogDate   |SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|
+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+
|1196192316    |3157        |2018-08-01|1         |4                  |NULL                  |13        |3              |3                |NULL            |
|1196192317    |3157        |2018-08-01|2         |NULL               |NULL                  |NULL      |1              |NULL             |NULL            |
|1196192318    |3157        |2018-08-01|3         |NULL               |NULL                  |NULL      |1              |NULL             |NULL            |
+--------------+------------+----------+----------+-------

`withColumn` agrega la(s) columna(s) a diferencia de `select` que solo crea un nuevo df con las nuevas columnas

In [10]:
logs = logs.withColumn(
    "Duration_seconds", (
        F.col("Duration").substr(1, 2).cast("int") * 3600
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
        ),
        )

In [11]:
split_table(logs, 3, 3)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+
|BroadcastLogID|LogServiceID|LogDate   |SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|
+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+
|1196192316    |3157        |2018-08-01|1         |4                  |NULL                  |13        |3              |3                |NULL            |NULL           |
|1196192317    |3157        |2018-08-01|2         |NULL               |NULL                  |NULL      |1              |NULL             |NULL            |NULL           |
|1196192318    |3157        |2018-08-01|3         |NULL               |NULL                  |NULL      |1              |NULL          

Pasar todos los nombres de columna a minuscula

In [12]:
def to_snake_case(word):
    return (''
            .join(
                ['_'+ i.lower() if i.isupper() else i for i in word]
                )
            .replace("_i_d", "_id")
            .replace("_n_o", "_no")
            .lstrip('_'))

In [13]:
logs = logs.toDF(*[to_snake_case(x) for x in logs.columns])
logs = logs.select(sorted(logs.columns))

In [14]:
split_table(logs, 3, 3)

+----------------------+-------------------------+----------------+-------------------------+-----------+-----------------+--------------+--------------------+-------------------+----------------+----------------+
|audience_target_age_id|audience_target_ethnic_id|broadcast_log_id|broadcast_origin_point_id|category_id|closed_caption_id|composition_id|country_of_origin_id|dub_drama_credit_id|duration        |duration_seconds|
+----------------------+-------------------------+----------------+-------------------------+-----------+-----------------+--------------+--------------------+-------------------+----------------+----------------+
|4                     |NULL                     |1196192316      |NULL                     |13         |3                |NULL          |3                   |NULL               |02:00:00.0000000|7200            |
|NULL                  |NULL                     |1196192317      |NULL                     |NULL       |1                |NULL          |NULL  

In [15]:
logs.describe().toPandas().set_index('summary').T

summary,count,mean,stddev,min,max
audience_target_age_id,16112,3.4929245283018866,1.0415963394745122,1,4
audience_target_ethnic_id,1710,120.56432748538012,71.98694059436134,4,337
broadcast_log_id,238945,1.2168651122760174E9,1.4969134241431221E7,1195788151,1249431576
broadcast_origin_point_id,9978,2.1390058127881337,0.9323192037553315,1,3
category_id,25506,18.485297577040697,9.655852252020837,1,29
closed_caption_id,224117,1.0316174141185184,0.24947032900378902,1,3
composition_id,9978,3.4141110442974543,0.9338603403654636,1,4
country_of_origin_id,17822,4.06390977443609,2.6159675821007315,2,11
dub_drama_credit_id,263,5.1254752851711025,3.451329711438564,1,10
duration,236724,None,None,00:00:01.0000000,06:30:09.0000000
